In [1]:
!pip install sentence_transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylin

In [5]:
import numpy as np
from sentence_transformers import SentenceTransformer, util
import faiss
import pandas as pd
# Подготовка данных
data = pd.read_csv('/content/books_data2.csv')
data['annotation'] = data['annotation'].astype(str)
data = data[data['annotation'].str.len() > 10]


In [6]:
# Загрузка модели
embedder = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
book_embeddings = embedder.encode(data['annotation'].tolist(), show_progress_bar=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/155 [00:00<?, ?it/s]

In [10]:
d = book_embeddings.shape[1]
index = faiss.IndexFlatIP(d)

book_embeddings = book_embeddings / np.linalg.norm(book_embeddings, axis=1)[:, None]
index.add(book_embeddings)

def search_books(query, k):
    query_embedding = embedder.encode([query])
    query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1)

    D, I = index.search(query_embedding, k)

    for i in range(k):
        book_info = f"Book: {data.iloc[I[0][i]]['title']}, Author: {data.iloc[I[0][i]]['annotation']}"
        similarity_score = D[0][i]
        print(f"Similarity: {similarity_score:.2f} - {book_info}")


search_books("русской литературы — роман в стихах А.С. Пушкина «Евгений Онегин» (1823–1831). История, покорившая мир, сопровождающая многие поколения читателей ", 5)

Similarity: 0.87 - Book: Евгений Онегин, Author: Классика и одно из самых значительных произведений русской литературы — роман в стихах А.С. Пушкина «Евгений Онегин» (1823–1831). История, покорившая мир, сопровождающая многие поколения читателей и пережившая бесчисленное количество интерпретаций и экранизаций, одной из которых стал
фильм Сарика Андреасяна «Онегин». Роман охватывает события с 1819 по 1825 год: от заграничных походов русской армии после разгрома Наполеона до восстания декабристов, и через все повествование тянется красная нить истории великой любви.
Similarity: 0.84 - Book: Евгений Онегин. Поэмы, Author: Александр Сергеевич Пушкин (1799 -1837) - великий русский поэт, создатель новой литературы, достигший необычайной легкости языка, изысканности и точности выражения мысли.
В сборник вошли его избранные произведения, составляющие золотой фонд классической русской литературы. 
Роман в стихах "Евгений Онегин" - энциклопедия пушкинской эпохи и культовая история несчастной люб

In [11]:
import pickle
with open('mpnet_base_embeddings.pkl', 'wb') as f:
    pickle.dump(book_embeddings, f)

faiss.write_index(index, 'mpnet_base_index.index')